In [2]:
old_lst = [1, 2, 3, 4, 5, 6, 7]
new_lst = [*old_lst, 10, 15, 20]
new_lst

[1, 2, 3, 4, 5, 6, 7, 10, 15, 20]

In [4]:
old_lst = [1, 2, 3, 4, 5, 6, 7]
new_lst = old_lst + [10, 15, 20]
new_lst

[1, 2, 3, 4, 5, 6, 7, 10, 15, 20]

In [1]:
import pandas as pd
from env import user, host, password, get_connection


In [3]:
url = get_connection(user, password, host, "tidy_data")
query = ''' 
SELECT * 
FROM sales
'''


In [4]:
sales_df = pd.read_sql(query, url)


In [5]:
# let's check the dataframe imported correctly 
sales_df.head()

,Product,2016 Sales,2016 PPU,2017 Sales,2017 PPU,2018 Sales,2018 PPU
0,A,673,5,231,7,173,9
1,B,259,3,748,5,186,8
2,C,644,3,863,5,632,5
3,D,508,9,356,11,347,14


In [ ]:
# Initial observations from the "sales" dataset include:
# Multi-column names (e.g., 2016 Sales...could be split into: "2016" and "Sales")
# Year, Sale, and PPU can be made into three (3) distinct columns

In [6]:
# let's check the shape of the dataset and subseuently, the info. 

sales_df.shape 
# 4 rows by
# 7 column dataset

(4, 7)

In [7]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Product     4 non-null      object
 1   2016 Sales  4 non-null      int64 
 2   2016 PPU    4 non-null      int64 
 3   2017 Sales  4 non-null      int64 
 4   2017 PPU    4 non-null      int64 
 5   2018 Sales  4 non-null      int64 
 6   2018 PPU    4 non-null      int64 
dtypes: int64(6), object(1)
memory usage: 352.0+ bytes


In [8]:
sales_df.head()

,Product,2016 Sales,2016 PPU,2017 Sales,2017 PPU,2018 Sales,2018 PPU
0,A,673,5,231,7,173,9
1,B,259,3,748,5,186,8
2,C,644,3,863,5,632,5
3,D,508,9,356,11,347,14


In [14]:
# let's first mels the table; keeping the "Product" column as the id_vars parameter

2016 Sales    4
2016 PPU      3
2017 Sales    4
2017 PPU      3
2018 Sales    4
2018 PPU      4
dtype: int64